In [ ]:
import keras # Должны быть версии 2.2.5
import tensorflow as tf # Должна быть версия 1.15.0
import scipy # Должна быть 1.2.1

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

print('Keras: ',keras.__version__)
print('TensorFlow:', tf.__version__)
print('Scipy:' ,scipy.__version__)

In [ ]:
# Загружаем MNIST
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Нормализуем значения пикспелей (от 0 до 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Приводим лейблы к формату One Hot
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Функция создания модели
def create_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
  return model

# Создаём модель
model = create_model()
model.summary()

# Обучаем модель
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test))

# Результат модели
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Сохраняем саму модель
# Сериализация в JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# Сохраняем(сериализуем) веса  в формате HDF5
model.save_weights("model.h5")
print("Сохранено")

In [ ]:
from flask import Flask, request # request - это объекты, которыми управляет flask handles (get, set, post и т.д.)
from flask import render_template # этот объект автоматически зарендерит нам шаблон html
import imageio # для работы с изображениями
import numpy as np
import keras.models
import re # регулярные питоновские выражения
from keras.models import model_from_json
import sys 
import os
# from load import * # наследуем все классы

# Задаем имя серверу
app = Flask(__name__)

import tensorflow as tf
import base64 # универсальный формат кодирования (часто используется с изображениями)
# from scipy.misc import imresize

# Открываем json файл разметки модели
json_file = open('model.json','r') 
loaded_model_json = json_file.read() # считываем
json_file.close() # закрываем
loaded_model = model_from_json(loaded_model_json) # используем керас, чтобы считать разметку архитектуры
loaded_model.summary()

loaded_model.load_weights("model.h5") # подгружаем веса
print("Загружено с диска")

# компилируем модель
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# оценим ее точность еще раз
loss,accuracy = loaded_model.evaluate(x_test,y_test)
print('accuracy:', accuracy)

In [ ]:
# инициализируем граф
global graph
graph = tf.compat.v1.get_default_graph()

In [ ]:
# заносим его в дефолтный граф для сессии
session = tf.compat.v1.Session(graph=graph)  
init_g = tf.compat.v1.global_variables_initializer()

In [ ]:
# декодируем изображение из сырого(бинарного) представления в его base64 
def convertImage(imgData1):
    imgstr = re.search(b'base64,(.*)',imgData1).group(1) # находим строчку, в которой содержится побитовое представление изображение,
                                                        # возвращает вторую(1) подгруппу
    with open('output.png','wb') as output: # временный файл, в который записываем декодированное представление
        output.write(base64.b64decode(imgstr))

In [ ]:
# # Указываем адрес по которому будет откликаться Web сервис и задаем саму функцию веб сервиса. Это наша стартовая страница
@app.route('/')
@app.route('/index')
def index():
    # рендерим заранее построенный HTML файл
    return render_template("index1.html")

In [ ]:
# маршрут, по которому будет идти предсказание
@app.route('/predict/',methods=['GET','POST'])
def predict():
    # в любой момент, когда пользователь вызовет predict
    # после того, как нарисовал картинку, произойдет
    # inference/внедрение, вернется результат классификации
    
    # получить сырой формат изображения (этот метод объекта Flask)
    imgData = request.get_data()
    
    # кодируем в подходящий формат
    convertImage(imgData)
    print ("debug")
    # считываем изображение в память
    x = imageio.imread('output.png',pilmode='L')
    # побитово инвертируем (из черного в белое и наоборот)
    x = np.invert(x)
    # выравниваем по размеру
    x = imresize(x,(28,28))
    
    # преобразуем в четырехразмерный тензор для модели
    x = x.reshape(1,28,28,1)
    print ("debug2")
    
    # инициализируем все переменные для графа
    session.run(init_g)
    #tf.compat.v1.global_variables_initializer()

   # session = tf.Session(graph=graph)
    
    # в сессии, заданной по умолчанию
    with session.as_default():
        # в графе, заданном по умолчанию
        with graph.as_default():
           
        # предсказать
            out = loaded_model.predict(x)            
            print(out)
            print(np.argmax(out,axis=1))
            print ("debug3")
            
        # преобразуем ответ в строку
            response = np.array_str(np.argmax(out,axis=1))
            return response
        # обнуляем граф, чтобы можно было ввести новые данные
        tf.reset_default_graph()

In [ ]:
if __name__ == "__main__":
    # этот порт выделяем под прогонку
    port = int(os.environ.get('PORT', 5002))
    
    # прогоняем на локальном сервере по заданному порту
    app.run(host='127.0.1.1', port=port)